## データの取得・確認

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path('__file__').resolve().parent.parent.parent.parent.parent))

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import model_selection
from termcolor import cprint

from notebooks.algorithms.scratch.utils.evaluation import score

cancer = datasets.load_breast_cancer()
cancer_data = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
cancer_target = pd.Series(data=cancer.target, name='target')

X_train, X_test, y_train, y_test = model_selection.train_test_split(cancer_data, cancer_target)

cprint('Cancer Data (samples: {}, features: {})'.format(*cancer_data.shape), 'blue', attrs=['bold'])
cprint('Cancer Target (samples: {})'.format(*cancer_target.shape), 'blue', attrs=['bold'])


Cancer Data (samples: 569, features: 30)
Cancer Target (samples: 569)


In [2]:
cancer_data.head(3)


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [3]:
cancer_target.unique()


array([0, 1])

## データ前処理

In [4]:
def standardize(X):
    """標準化する.
    
    X: データセット
    """
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)


def add_ones(X):
    """特徴量の0列目に1を加える.
    
    X: データセット
    """
    X['0'] = 1
    return X.sort_index(axis=1, ascending=True)


X_train_std = standardize(X_train)
X_test_std = standardize(X_test)

# バイアス項を追加
X_train_std = add_ones(X_train_std)
X_test_std = add_ones(X_test_std)

X_train_std.head(3)


,0,area error,compactness error,concave points error,concavity error,fractal dimension error,mean area,mean compactness,mean concave points,mean concavity,...,worst area,worst compactness,worst concave points,worst concavity,worst fractal dimension,worst perimeter,worst radius,worst smoothness,worst symmetry,worst texture
285,1,-0.397555,-0.948408,-1.369454,-0.928978,-0.677762,-0.444821,-1.168106,-1.178986,-1.081482,...,-0.525479,-1.199099,-1.585840,-1.275728,-1.148608,-0.618151,-0.543184,-1.229831,-0.650781,-0.416370
214,1,-0.190781,0.639810,0.712410,0.147776,0.347442,-0.089690,0.537003,0.472244,0.331691,...,-0.093649,1.012257,1.013654,0.527466,1.091404,0.264504,0.154360,1.050277,2.991253,1.453347
383,1,-0.476153,0.637080,-0.186102,0.476460,1.135430,-0.520983,0.519651,-0.486442,-0.346024,...,-0.461892,0.708868,-0.208090,0.264433,1.629615,-0.328230,-0.402014,0.472592,-0.135415,-0.408428


## モデルの構築・訓練

In [5]:
class LogisticRegression(object):
    """勾配降下法に基づくロジスティック回帰."""

    def __init__(self, alpha=0.01, eps=1e-6):
        """
        ---パラメータ-------
        alpha: 学習率
        eps: 収束の目安
        --------------------
        
        ---属性-------------
        _w: パラメータ（重み）
        --------------------
        """
        self.alpha = alpha
        self.eps = eps

        self._w = 0

    def fit(self, X, y):
        """訓練データで学習する.
        
        ---パラメータ-------
        X: 訓練データセット
        y: ターゲット
        --------------------
        
        """
        # パラメータの初期値はランダムに設定する
        theta = np.random.rand(X.shape[1])
        error = self.J(X, y, theta)
        count = 0
        diff = 1

        # コスト関数の値が殆ど変動しなくなったら収束とする
        while diff > self.eps:
            # パラメータの更新処理
            grad = (1 / len(y)) * np.dot(self.h(X, theta) - y, X)
            theta = theta - self.alpha * grad

            # 収束状況を確認するための処理
            current_error = self.J(X, y, theta)
            diff = error - current_error
            error = current_error
            count += 1
            print('\r【{}回目】Error: {}, Diff: {}, Grad: {}'.format(count, error, diff, grad.sum()), end='')

        self._w = theta
        return self

    def predict(self, X):
        """クラスラベルを予測する.
        
        X: テストデータセット
        """
        return np.where(self.h(X, self._w) >= 0.5, 1, 0)

    def activate(self, z):
        """ロジスティック関数（活性化関数）.
        
        z: 仮説関数の予測値
        """
        return 1 / (1 + np.exp(-z))

    def h(self, X, theta):
        """仮説関数.
        
        X: 訓練データセット
        theta: パラメータ（重み）
        """
        return self.activate(np.dot(X, theta))

    def J(self, X, y, theta):
        """目的関数.
        
        X: 訓練データセット
        y: ターゲット
        theta: パラメータ（重み）
        """
        delta = 1e-7  # np.log()に0が混在しないようにする
        return - (1 / len(y)) * (
            np.sum(y * np.log(self.h(X, theta) + delta) + (1 - y) * np.log(1 - self.h(X, theta) + delta)))


In [6]:
lr = LogisticRegression()
lr.fit(X_train_std, y_train)


【113回目】Error: 0.47523428557541375, Diff: 0.009036956988139944, Grad: 4.940351319771515

【225回目】Error: 0.1977905955929122, Diff: 0.0006256243796311511, Grad: 1.231192383617765

【337回目】Error: 0.15682017210095905, Diff: 0.00022460876243163286, Grad: 0.6579707372007102

【452回目】Error: 0.1376335343655377, Diff: 0.000126804602576025, Grad: 0.443273062622185

【565回目】Error: 0.12586690273296783, Diff: 8.637361789889941e-05, Grad: 0.3384249368065848

...

【2154回目】Error: 0.08152549734588227, Diff: 9.985201435913638e-06, Grad: 0.09283015128546228

【2270回目】Error: 0.08041524947232033, Diff: 9.182214921896192e-06, Grad: 0.08851000416832641

【2386回目】Error: 0.07939210833279316, Diff: 8.479406673228551e-06, Grad: 0.08458852971953194

【2483回目】Error: 0.07859566669893193, Diff: 7.956496872907337e-06, Grad: 0.08157628414585193

【9043回目】Error: 0.06083968821575812, Diff: 9.999302095289875e-07, Grad: 0.02517247616512697474

## 予測・評価

In [7]:
predict = pd.DataFrame(lr.predict(X_train_std), columns=['TARGET'])
predict.head(10)

,TARGET
0,1
1,0
2,1
3,0
4,1
5,1
6,1
7,0
8,1
9,0


In [8]:
train_score = score(predict, y_train)
test_score = score(pd.DataFrame(lr.predict(X_test_std), columns=['target']), y_test)
cprint('train_score: {}\ntest_score: {}'.format(train_score, test_score), 'red', attrs=['bold'])

train_score: 0.9882629107981221
test_score: 0.958041958041958
